In [7]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn

from transformers import BertTokenizer, BertModel, AutoTokenizer
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split



In [2]:
os.getcwd()
# change the path to C:\Users\Kyriakos\Desktop\MsC AI\NLP\NLP AirBNB Project\AirBnB_score_prediction\dataset
# os.chdir('C:\\Users\\Kyriakos\\Desktop\\MsC AI\\NLP\\NLP AirBNB Project\\AirBnB_score_prediction\\dataset')
# os.getcwd()


'g:\\.shortcut-targets-by-id\\1CZZx9Bpz7hB-BhPFsqAL3fRVyxM1XfwF\\AirBnB_project'

In [8]:
# 'listings_comments.csv is the datset compined 

# reaf the datset from Google Drive G:\.shortcut-targets-by-id\1CZZx9Bpz7hB-BhPFsqAL3fRVyxM1XfwF\AirBnB_project
data = pd.read_csv('G:\\.shortcut-targets-by-id\\1CZZx9Bpz7hB-BhPFsqAL3fRVyxM1XfwF\\AirBnB_project\\listings_comments.csv')


### check  for nan values


In [9]:
print(f'Description nan values: {data.description.isna().sum()}')
print(f'Comments nan values: {data.neighborhood_overview.isna().sum()}')
print(f'Comments nan values: {data.neighbourhood.isna().sum()}')

Description nan values: 6
Comments nan values: 2492
Comments nan values: 2492


In [10]:
def fill_nan(df, column):
    df[column].fillna('UNK', inplace=True)
    return df

data = fill_nan(data, 'description')
data = fill_nan(data, 'neighborhood_overview')
data = fill_nan(data, 'neighbourhood')

In [11]:
print(f'Description nan values: {data.description.isna().sum()}')
print(f'Comments nan values: {data.neighborhood_overview.isna().sum()}')
print(f'Comments nan values: {data.neighbourhood.isna().sum()}')

Description nan values: 0
Comments nan values: 0
Comments nan values: 0


In [12]:
# check if there are any missing values in "review_scores_rating"
print(data['review_scores_rating'].isnull().sum())

# fill missing values with the mean
data['review_scores_rating'].fillna(data['review_scores_rating'].mean(), inplace=True)

print(data['review_scores_rating'].isnull().sum())

682
0


In [13]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
# Remove unnecessary columns
columns_to_remove = ['listing_url', 'scrape_id', 'last_scraped', 'source', 'picture_url', 'host_url', 'host_name', 
                     'host_location', 'host_thumbnail_url', 'host_picture_url', 'host_verifications', 
                     'host_has_profile_pic', 'host_identity_verified', 'neighbourhood_group_cleansed', 
                     'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights',
                     'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability',
                     'availability_30', 'availability_60', 'availability_90', 'availability_365', 
                     'calendar_last_scraped', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 
                     'first_review', 'last_review', 'license', 'instant_bookable', 'reviews_per_month']


In [15]:
data.drop(columns_to_remove, axis=1, inplace=True)

# Remove rows with missing review_scores_rating values
# data.dropna(subset=['review_scores_rating'], inplace=True)

# we can do it later when we combine all embeddings in one dataframe
# Split into training and validation sets
X = data.drop(['review_scores_rating'], axis=1)
y = data['review_scores_rating']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


## Function that takes a column and returns an embeddings matrix

In [16]:

# endoded_descriptions = []

# for description in df.description:
#     # Tokenize text and add special tokens
#     encoded = tokenizer.encode_plus(
#         description,
#         add_special_tokens=True,
#         max_length=64,  # limit the length of the encoded text to 64 tokens
#         pad_to_max_length=True,  # pad shorter texts and truncate longer texts
#         return_attention_mask=True,
#         return_tensors='pt'  # return PyTorch tensors
#     )
#     # Add encoded listing to the list
#     endoded_descriptions.append(encoded)

# # Convert encoded listings to PyTorch tensors
# input_ids = torch.cat([x['input_ids'] for x in endoded_descriptions])
# attention_mask = torch.cat([x['attention_mask'] for x in endoded_descriptions])

# # Pass the input through the BERT model to get the output
# with torch.no_grad():
#     outputs = model(input_ids, attention_mask)

# # Extract the output embeddings for each listing
# descriptions_embeddings = outputs[0][:, 0, :]
# # print(descriptions_embeddings.shape)


# def get_embeddings(listings, model, tokenizer):
#     # Pass the input through the BERT model to get the output
#     with torch.no_grad():
#         encoded_listings = []
#         for listing in listings:
#             # Tokenize text and add special tokens
#             encoded = tokenizer.encode_plus(
#                 listing,
#                 add_special_tokens=True,
#                 max_length=64,  # limit the length of the encoded text to 64 tokens
#                 pad_to_max_length=True,  # pad shorter texts and truncate longer texts
#                 return_attention_mask=True,
#                 return_tensors='pt'  # return PyTorch tensors
#             )
#             # Add encoded listing to the list
#             encoded_listings.append(encoded)

#         # Convert encoded listings to PyTorch tensors
#         input_ids = torch.cat([x['input_ids'] for x in encoded_listings]).to(model.device)
#         attention_mask = torch.cat([x['attention_mask'] for x in encoded_listings]).to(model.device)
        
#         outputs = model(input_ids=input_ids, attention_mask=attention_mask)

#         # Extract the output embeddings for each listing
#         listings_embeddings = outputs[0][:, 0, :]
#         return listings_embeddings.cpu()








In [ ]:

# check if cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
# move the model to the device
model.to(device)

# parallelize the model across multiple GPUs
model = nn.DataParallel(model)

Device: cpu


NameError: name 'model' is not defined

In [20]:


# def get_embeddings(listings):
        

   
    # def encode(listings):
    #     for listing in listings:
    #         # Tokenize text and add special tokens
    #         encoded = tokenizer.encode_plus(
    #             listing,
    #             add_special_tokens=True,
    #             max_length=64,  # limit the length of the encoded text to 64 tokens
    #             pad_to_max_length=True,  # pad shorter texts and truncate longer texts
    #             return_attention_mask=True,
    #             return_tensors='pt'  # return PyTorch tensors
    #         )
    #         yield encoded

    # # Convert encoded listings to PyTorch tensors
    # input_ids = []
    # attention_mask = []
    # for encoded_listing in encode(listings):
    #     input_ids.append(encoded_listing['input_ids'])
    #     attention_mask.append(encoded_listing['attention_mask'])

    # input_ids = torch.cat(input_ids).to(device)
    # attention_mask = torch.cat(attention_mask).to(device)
    # # set the batcxh size
    # batch_size = 32

    # #  Split the input_ids and attention_mask into batches
    # input_ids_batches = torch.split(input_ids, batch_size)
    # attention_mask_batches = torch.split(attention_mask, batch_size)

    # # Pass the input through the BERT model to get the output

    # embeddings = []
    # with torch.no_grad():
    #     outputs = []
    #     for input_ids_batch, attention_mask_batch in zip(input_ids_batches, attention_mask_batches):
    #         outputs.append(model(input_ids=input_ids_batch, attention_mask=attention_mask_batch))

    #         embeddings.append(outputs[0][0][:, 0, :])

    # # clear the memory of the GPU deticated memory usage
    # torch.cuda.empty_cache()
    # del outputs
    # del input_ids
    # del attention_mask

    # # Extract the output embeddings for each listing
    # listings_embeddings = torch.cat(embeddings)
    # return listings_embeddings.to('cpu')

model = SentenceTransformer('all-MiniLM-L6-v2')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
# move the model to the device
model.to(device)

# parallelize the model across multiple GPUs
model = nn.DataParallel(model) 

# apply it with sbert model

def get_embeddings(listings):

    # torch.tensor(listings).to(device)
    
    # set the batch size
    batch_size = 32

    #  Split the input_ids and attention_mask into batches
    listings_batches = torch.split(listings, batch_size)

    # Pass the input through the BERT model to get the output

    embeddings = []
    with torch.no_grad():
        outputs = []
        for listings_batch in listings_batches:
            outputs.append(model.encode(listings_batch))

            embeddings.append(outputs[0])

    # clear the memory of the GPU deticated memory usage
    torch.cuda.empty_cache()
    del outputs

    # Extract the output embeddings for each listing
    listings_embeddings = torch.cat(embeddings)
    return listings_embeddings.to('cpu')

Device: cpu


In [39]:
train_descriptions_embeddings = get_embeddings(X_train.description)
val_descriptions_embeddings = get_embeddings(X_val.description)

# Convert the embeddings to numpy arrays
train_descriptions_embeddings = train_descriptions_embeddings.numpy()   
val_descriptions_embeddings = val_descriptions_embeddings.numpy()

# Create a dataframe from the embeddings
train_descriptions_embeddings = pd.DataFrame(train_descriptions_embeddings)
val_descriptions_embeddings = pd.DataFrame(val_descriptions_embeddings)


In [40]:
descriptions_embeddings.shape   

torch.Size([7008, 768])

In [21]:
# neighbourhood_overview_embeddings = get_embeddings(data.neighborhood_overview.values)
neighbourhood_overview_embeddings = get_embeddings(torch.tensor(data.neighborhood_overview.values))


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [29]:
neighbourhood_emmbeddings = get_embeddings(data.neighbourhood)

c:\Users\kyriakos\anaconda3\envs\nlp1\lib\site-packages\transformers\tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [35]:
neighbourhood_emmbeddings.shape

torch.Size([7008, 768])

In [38]:
name_embeddings = get_embeddings(data.name)
name_embeddings.shape

c:\Users\kyriakos\anaconda3\envs\nlp1\lib\site-packages\transformers\tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


torch.Size([7008, 768])

In [82]:
data.shape

(6998, 41)

In [34]:
pd.set_option('display.max_columns', None)
data.head()
# data.shape

,listing_id,name,description,neighborhood_overview,host_id,host_since,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,neighbourhood,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,comments
0,2818,Quiet Garden View Room & Super Fast Wi-Fi,Quiet Garden View Room & Super Fast Wi-Fi<br /...,"Indische Buurt (""Indies Neighborhood"") is a ne...",3159,2008-09-24,"Upon arriving in Amsterdam, one can imagine as...",within an hour,100%,100%,t,Indische Buurt,1,1,"Amsterdam, North Holland, Netherlands",Oostelijk Havengebied - Indische Buurt,52.36435,4.94358,Private room in rental unit,Private room,2,NaN,1.5 shared baths,1.0,2.0,"[""Drying rack for clothing"", ""Host greets you""...",$69.00,3,28,4.89,4.93,4.99,4.97,4.98,4.69,4.81,1,0,1,0,['Daniel is really cool. The place was nice an...
1,311124,*historic centre* *bright* *canal view* *jordaan*,> Please be so kind to book ONLY AFTER conta...,Perfect location in the lively centre. All his...,1600010,2012-01-12,"We love cycling, travelling, healthy food and ...",within a few hours,100%,68%,f,Jordaan,1,1,"Amsterdam, North Holland, Netherlands",Centrum-West,52.37104,4.87713,Entire rental unit,Entire home/apt,2,NaN,1 bath,1.0,1.0,"[""Drying rack for clothing"", ""Wifi"", ""Waterfro...",$325.00,5,15,4.76,4.72,4.79,4.90,4.92,4.93,4.60,1,1,0,0,['Nice apartment in a beautifull área of Amste...
2,319985,Elegant Appartement Central Location.,"From December 26 thru January 2, minimum stay ...",Bij aankomst vertel ik mijn gasten graag over ...,1640702,2012-01-23,I like to welcome guests who take good care of...,within a day,93%,89%,t,De Pijp,2,3,"De Pijp, North Holland, Netherlands",De Pijp - Rivierenbuurt,52.35669,4.90187,Entire rental unit,Entire home/apt,4,NaN,1 bath,1.0,2.0,"[""Building staff"", ""Wifi"", ""Hair dryer"", ""Bed ...",$244.00,1,90,4.75,4.83,4.69,4.86,4.78,4.85,4.62,2,1,1,0,"['Einfach große Klasse! Zentrale Lage, Einkauf..."
3,327285,beautiful designed ap.+bikes+garden,Cosy and comfortable apartment on the ground f...,'De Pijp' as we Amsterdammers know it today is...,1672823,2012-01-30,"We are the parents of 4 beautiful children, tw...",within an hour,100%,100%,f,De Pijp,3,4,"Amsterdam, North Holland, Netherlands",De Pijp - Rivierenbuurt,52.35625,4.89323,Private room in home,Private room,4,NaN,1.5 baths,1.0,6.0,"[""Board games"", ""Drying rack for clothing"", ""W...",$244.00,2,30,4.94,4.91,4.92,4.98,4.99,4.91,4.84,2,0,2,0,"[""This was by far the best Airbnb experience I..."
4,331946,Lovely FAMILY house near Vondelpark,* Our house is only suitable for a couple or f...,UNK,1687595,2012-02-02,"Welcome guests, I'm Sebastiaan and it would be...",NaN,NaN,NaN,f,Oud-West,1,1,UNK,De Baarsjes - Oud-West,52.36060,4.85739,Entire loft,Entire home/apt,4,NaN,1 bath,4.0,4.0,"[""Host greets you"", ""Wifi"", ""Children\u2019s d...",$220.00,2,14,4.88,4.88,4.64,4.92,4.88,4.79,4.75,1,1,0,0,"['Amazing house, perfect location and a dream ..."


In [85]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

## neighbourhood_cleansed column

In [86]:
data.neighbourhood_cleansed.value_counts()

De Baarsjes - Oud-West                    1139
Centrum-West                               922
De Pijp - Rivierenbuurt                    802
Centrum-Oost                               681
Westerpark                                 481
Zuid                                       464
Oud-Oost                                   408
Bos en Lommer                              344
Oud-Noord                                  322
Oostelijk Havengebied - Indische Buurt     260
Watergraafsmeer                            208
Noord-West                                 179
IJburg - Zeeburgereiland                   157
Slotervaart                                143
Noord-Oost                                 113
Geuzenveld - Slotermeer                     87
Buitenveldert - Zuidas                      80
De Aker - Nieuw Sloten                      49
Bijlmer-Centrum                             44
Gaasperdam - Driemond                       44
Osdorp                                      43
Bijlmer-Oost 

In [87]:
# label encode the neighbourhood_cleansed column
data['neighbourhood_cleansed'] = le.fit_transform(data['neighbourhood_cleansed'])

In [88]:
data.neighbourhood_cleansed.value_counts()

7     1139
5      922
8      802
4      681
20     481
21     464
17     408
2      344
16     322
14     260
19     208
13     179
11     157
18     143
12     113
10      87
3       80
6       49
0       44
9       44
15      43
1       28
Name: neighbourhood_cleansed, dtype: int64

## Column property_type

In [ ]:
# i have used label ecoder but we can try embedding for this column as well

In [90]:
data.property_type.value_counts()

Entire rental unit                    2540
Entire condo                          1304
Private room in rental unit            490
Entire home                            471
Private room in bed and breakfast      290
Entire townhouse                       213
Entire loft                            187
Private room in condo                  148
Houseboat                              130
Private room in home                   128
Room in hotel                          115
Room in boutique hotel                 111
Private room in houseboat               97
Boat                                    93
Private room in guest suite             92
Private room in townhouse               86
Entire serviced apartment               82
Private room in boat                    56
Private room in loft                    43
Room in bed and breakfast               26
Entire villa                            25
Shared room in hostel                   23
Entire guest suite                      23
Room in apa

In [91]:
# values less or equal to 4 will be replaced with 'other'
data['property_type'] = data['property_type'].apply(lambda x: 'other' if x in data.property_type.value_counts()[data.property_type.value_counts() <= 10].index else x)

In [92]:
data.property_type.value_counts()

Entire rental unit                    2540
Entire condo                          1304
Private room in rental unit            490
Entire home                            471
Private room in bed and breakfast      290
Entire townhouse                       213
Entire loft                            187
Private room in condo                  148
Houseboat                              130
Private room in home                   128
Room in hotel                          115
Room in boutique hotel                 111
Private room in houseboat               97
other                                   94
Boat                                    93
Private room in guest suite             92
Private room in townhouse               86
Entire serviced apartment               82
Private room in boat                    56
Private room in loft                    43
Room in bed and breakfast               26
Entire villa                            25
Entire guest suite                      23
Shared room

In [93]:
# label encode the property_type column
data['property_type'] = le.fit_transform(data['property_type'])


In [94]:
data.room_type.value_counts().__len__

<bound method Series.__len__ of Entire home/apt    5154
Private room       1743
Hotel room           62
Shared room          39
Name: room_type, dtype: int64>

## Column host_neighbourhood

In [41]:
data.host_neighbourhood.value_counts()

Oud-West                               553
Grachtengordel                         244
De Pijp                                234
Jordaan                                189
Oosterparkbuurt                        157
Bos en Lommer                          126
Indische Buurt                         116
Oost                                    84
Nieuwmarkt en Lastage                   79
Rivierenbuurt                           78
Hoofddorppleinbuurt                     69
Watergraafsmeer                         64
De Wallen                               62
Oostelijke Eilanden en Kadijken         57
Westelijke Eilanden                     57
Amsterdam Centrum                       56
Weesperbuurt en Plantage                55
Frederik Hendrikbuurt                   53
Zeeburg                                 52
Oud-Zuid                                50
Stadionbuurt                            45
Spaarndammer en Zeeheldenbuurt          45
Museumkwartier                          45
Slotervaart

In [43]:
# value_counts of host_neighbourhood == 1 rename as "other"
vc = data.host_neighbourhood.value_counts()
single_neighborhoods = vc[vc == 1].index.tolist()
data.loc[data.host_neighbourhood.isin(single_neighborhoods), 'host_neighbourhood'] = 'other'



In [44]:
data.host_neighbourhood.value_counts()

Oud-West                               553
Grachtengordel                         244
De Pijp                                234
Jordaan                                189
Oosterparkbuurt                        157
Bos en Lommer                          126
Indische Buurt                         116
Oost                                    84
Nieuwmarkt en Lastage                   79
Rivierenbuurt                           78
Hoofddorppleinbuurt                     69
Watergraafsmeer                         64
De Wallen                               62
Westelijke Eilanden                     57
Oostelijke Eilanden en Kadijken         57
Amsterdam Centrum                       56
Weesperbuurt en Plantage                55
Frederik Hendrikbuurt                   53
Zeeburg                                 52
Oud-Zuid                                50
Spaarndammer en Zeeheldenbuurt          45
Museumkwartier                          45
Stadionbuurt                            45
Slotervaart

In [46]:
# apply label encoder to host_neighbourhood


data['host_neighbourhood'] = le.fit_transform(data['host_neighbourhood'])

## Column host_response_time

In [62]:
len(data.host_response_time.value_counts())

4

In [63]:
data.host_response_time.value_counts()

within an hour        2645
within a few hours    1123
within a day          1083
a few days or more     108
Name: host_response_time, dtype: int64

In [66]:
# convert to category with label encoding
data['host_response_time'] = le.fit_transform(data['host_response_time'])



In [71]:
print(data['host_response_time'].value_counts())
print(data['host_response_time'])

3    2645
4    2039
2    1123
1    1083
0     108
Name: host_response_time, dtype: int64
0       3
1       2
2       1
3       3
4       4
       ..
6993    4
6994    4
6995    4
6996    4
6997    2
Name: host_response_time, Length: 6998, dtype: int32


3377

## Column host_about

In [74]:
len(data.host_about.value_counts())

3377

In [ ]:
data.host_about.head()

0    Upon arriving in Amsterdam, one can imagine as...
1    We love cycling, travelling, healthy food and ...
2    I like to welcome guests who take good care of...
3    We are the parents of 4 beautiful children, tw...
4    Welcome guests, I'm Sebastiaan and it would be...
Name: host_about, dtype: object

In [78]:
# check if there is any null value
data.host_about.isnull().sum()

2915

In [79]:
# fill the with 'UNK'
data.host_about.fillna('UNK', inplace=True)

In [80]:
host_abput_embeddings = get_embeddings(data.host_about)
host_abput_embeddings.shape

c:\Users\kyriakos\anaconda3\envs\nlp1\lib\site-packages\transformers\tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


torch.Size([7008, 768])